In [5]:
import requests
import pandas as pd
from datetime import timezone

# 1. Obtener noticias desde la API

API_KEY = 'KEY'
url = f'https://newsapi.org/v2/top-headlines?language=en&pageSize=100&apiKey={API_KEY}'
response = requests.get(url)
data = response.json()

# 2. Convertir a DataFrame
df_news = pd.json_normalize(data['articles'])

# 3. Asegurar tipos de datos
df_news['publishedAt'] = pd.to_datetime(df_news['publishedAt'], utc=True, errors='coerce')
df_news['publishedAt_lima'] = df_news['publishedAt'].dt.tz_convert('America/Lima')

df_news.head()

,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name,publishedAt_lima
0,Tierney Sneed,Judge orders Trump administration to keep Sign...,A federal judge on Thursday ordered key Trump ...,https://www.cnn.com/2025/03/27/politics/signal...,https://media.cnn.com/api/v1/images/stellar/pr...,2025-03-27 23:22:00+00:00,A federal judge on Thursday ordered key Trump ...,cnn,CNN,2025-03-27 18:22:00-05:00
1,None,Trump tariffs: Can Vietnam golf its way out of...,The South East Asian country has the third lar...,https://www.bbc.com/news/articles/cge1r1ezw74o,https://ichef.bbci.co.uk/news/1024/branded_new...,2025-03-27 23:13:58+00:00,Annabelle Liang\r\nVietnam's PM has said he wo...,None,BBC News,2025-03-27 18:13:58-05:00
2,"Mark Bergin, Laura Leslie, Josie Zimmer",'We're going to feel the impacts': NC Health D...,The North Carolina Department of Health and Hu...,https://www.wral.com/story/we-re-going-to-feel...,https://images.wral.com/asset/news/local/2008/...,2025-03-27 23:07:16+00:00,Interim State Health Director Dr. Kelly Kimple...,None,WRAL.com,2025-03-27 18:07:16-05:00
3,"Sarah Ferris, Arit John, David Wright, Steve C...",Republicans grow concerned about Trump’s handp...,Top Republicans grew so concerned about the la...,https://www.cnn.com/2025/03/27/politics/randy-...,https://media.cnn.com/api/v1/images/stellar/pr...,2025-03-27 21:44:00+00:00,Top Republicans grew so concerned about the la...,cnn,CNN,2025-03-27 16:44:00-05:00
4,None,Six people dead after tourist submarine sinks ...,Another 39 people were rescued after the vesse...,https://www.bbc.com/news/articles/cx2erv557lgo,https://ichef.bbci.co.uk/news/1024/branded_new...,2025-03-27 21:42:20+00:00,Six Russian tourists have died after a tourist...,None,BBC News,2025-03-27 16:42:20-05:00


In [8]:
# 4. Indicadores de Calidad

# --- Completitud ---
completitud = {
    'Total noticias': len(df_news),
    'Nulos en título': df_news['title'].isnull().sum(),
    'Nulos en descripción': df_news['description'].isnull().sum(),
    'Nulos en fecha de publicación': df_news['publishedAt'].isnull().sum(),
    'Nulos en fuente': df_news['source.name'].isnull().sum(),
    'Porcentaje de noticias sin título': round(df_news['title'].isnull().mean() * 100, 2)
}

# --- Unicidad ---
unicidad = {
    'Títulos duplicados': df_news.duplicated('title').sum(),
    'URLs duplicadas': df_news.duplicated('url').sum(),
    '¿Títulos únicos = total?': df_news['title'].nunique() == len(df_news)
}

# --- Consistencia ---
consistencia = {
    'Noticias con título pero sin contenido': ((df_news['title'].notnull()) & (df_news['content'].isnull())).sum(),
    'Noticias con fecha futura': (df_news['publishedAt'] > pd.Timestamp.now(tz=timezone.utc)).sum(),
    'Fechas ordenadas cronológicamente': df_news['publishedAt'].is_monotonic_increasing
}

# --- Validez ---
validez = {
    'Formato de fecha válido': pd.api.types.is_datetime64_any_dtype(df_news['publishedAt']),
    'Mínima fecha publicada (UTC)': df_news['publishedAt'].min(),
    'Máxima fecha publicada (UTC)': df_news['publishedAt'].max()
}

# --- Actualidad ---
actualidad = {
    'Días únicos de publicación': df_news['publishedAt_lima'].dt.date.nunique(),
    'Cantidad de noticias por día promedio': round(len(df_news) / df_news['publishedAt_lima'].dt.date.nunique(), 2) if df_news['publishedAt_lima'].dt.date.nunique() else 0
}

# 5. Mostrar todos los indicadores

print("\nIndicadores de Calidad - Dataset de Noticias\n")

print("Completitud:")
for k, v in completitud.items():
    print(f" - {k}: {v}")

print("\nUnicidad:")
for k, v in unicidad.items():
    print(f" - {k}: {v}")

print("\nConsistencia:")
for k, v in consistencia.items():
    print(f" - {k}: {v}")

print("\nValidez:")
for k, v in validez.items():
    print(f" - {k}: {v}")

print("\nActualidad:")
for k, v in actualidad.items():
    print(f" - {k}: {v}")


Indicadores de Calidad - Dataset de Noticias

Completitud:
 - Total noticias: 33
 - Nulos en título: 0
 - Nulos en descripción: 1
 - Nulos en fecha de publicación: 0
 - Nulos en fuente: 0
 - Porcentaje de noticias sin título: 0.0

Unicidad:
 - Títulos duplicados: 0
 - URLs duplicadas: 0
 - ¿Títulos únicos = total?: True

Consistencia:
 - Noticias con título pero sin contenido: 2
 - Noticias con fecha futura: 0
 - Fechas ordenadas cronológicamente: False

Validez:
 - Formato de fecha válido: True
 - Mínima fecha publicada (UTC): 2025-03-25 15:39:45+00:00
 - Máxima fecha publicada (UTC): 2025-03-27 23:22:00+00:00

Actualidad:
 - Días únicos de publicación: 3
 - Cantidad de noticias por día promedio: 11.0
